In [2]:
pip install qrcode[pil]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import qrcode

# Create a QRCode object with custom settings
qr = qrcode.QRCode(
    version=1,  # controls the size of the QR Code (1 is 21x21)
    error_correction=qrcode.constants.ERROR_CORRECT_L,  # L: 7% error correction
    box_size=10,  # size of each box in pixels
    border=4,  # thickness of the border (default is 4)
)

# Add your data
qr.add_data("Tushar")
qr.make(fit=True)

# Create the image
img = qr.make_image(fill_color="darkblue", back_color="lightyellow")

# Save it
img.save("custom_qrcode.png")
print("Custom QR code generated successfully!")

Custom QR code generated successfully!


In [1]:
import time
import platform

print(f"System: {platform.system()} {platform.release()}")
print(f"Processor: {platform.processor()}")

# CPU test
start = time.time()
total = sum(range(50_000_000))  # Summing numbers
cpu_time = time.time() - start
print(f"CPU test time: {cpu_time:.2f} seconds")

# Memory test
start = time.time()
lst = [0] * 50_000_000  # Allocate large list
mem_time = time.time() - start
print(f"Memory allocation time: {mem_time:.2f} seconds")

# Simple score (lower time = better score)
score = 1000 / (cpu_time + mem_time)
print(f"Performance Score: {score:.2f}")


System: Windows 11
Processor: Intel64 Family 6 Model 170 Stepping 4, GenuineIntel
CPU test time: 2.37 seconds
Memory allocation time: 0.21 seconds
Performance Score: 387.81


In [2]:
import time
import platform
import os
import tempfile

print(f"System: {platform.system()} {platform.release()}")
print(f"Processor: {platform.processor()}")

# --- CPU Test ---
start = time.time()
total = sum(range(50_000_000))
cpu_time = time.time() - start
print(f"CPU test time: {cpu_time:.2f} seconds")

# --- Memory Test ---
start = time.time()
lst = [0] * 50_000_000
mem_time = time.time() - start
print(f"Memory allocation time: {mem_time:.2f} seconds")

# --- Disk Test ---
file_size_mb = 200
temp_file = os.path.join(tempfile.gettempdir(), "disk_benchmark.tmp")

# Write test
start = time.time()
with open(temp_file, "wb") as f:
    f.write(os.urandom(file_size_mb * 1024 * 1024))
write_time = time.time() - start

# Read test
start = time.time()
with open(temp_file, "rb") as f:
    f.read()
read_time = time.time() - start

# Remove test file
os.remove(temp_file)

disk_write_speed = file_size_mb / write_time
disk_read_speed = file_size_mb / read_time
print(f"Disk write speed: {disk_write_speed:.2f} MB/s")
print(f"Disk read speed: {disk_read_speed:.2f} MB/s")

# --- GPU Check ---
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"GPU detected: {gpu_name}")
    else:
        print("No GPU detected (or not accessible by PyTorch).")
except ImportError:
    print("PyTorch not installed — skipping GPU check.")

# --- Combined Performance Score ---
score = (1000 / (cpu_time + mem_time)) + (disk_read_speed + disk_write_speed) / 20
print(f"Performance Score: {score:.2f}")


System: Windows 11
Processor: Intel64 Family 6 Model 170 Stepping 4, GenuineIntel
CPU test time: 2.31 seconds
Memory allocation time: 0.30 seconds
Disk write speed: 260.07 MB/s
Disk read speed: 872.42 MB/s
PyTorch not installed — skipping GPU check.
Performance Score: 439.84


In [3]:
!pip install torch --quiet


^C


In [ ]:
import torch
import time

if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU detected: {gpu_name}")

    # Create two large random matrices on GPU
    size = 4000  # increase for more stress
    a = torch.rand((size, size), device=device)
    b = torch.rand((size, size), device=device)

    # Warm-up run (ignore timing)
    torch.matmul(a, b)

    # Timed run
    torch.cuda.synchronize()
    start = time.time()
    torch.matmul(a, b)
    torch.cuda.synchronize()
    gpu_time = time.time() - start

    # Simple GFLOPS estimate
    flops = 2 * (size ** 3) / gpu_time / 1e9
    print(f"GPU matrix multiply time: {gpu_time:.4f} sec")
    print(f"Estimated performance: {flops:.2f} GFLOPS")

else:
    print("No compatible GPU detected or CUDA not available.")


In [1]:
!pip install cupy-cuda12x --quiet

^C


In [ ]:
import cupy as cp
import time

try:
    # Check GPU
    gpu_name = cp.cuda.runtime.getDeviceProperties(0)["name"].decode("utf-8")
    print(f"GPU detected: {gpu_name}")

    # Create two big random matrices
    size = 4000
    a = cp.random.rand(size, size, dtype=cp.float32)
    b = cp.random.rand(size, size, dtype=cp.float32)

    # Warm-up
    cp.matmul(a, b)
    cp.cuda.Stream.null.synchronize()

    # Timed run
    start = time.time()
    cp.matmul(a, b)
    cp.cuda.Stream.null.synchronize()
    gpu_time = time.time() - start

    # Calculate GFLOPS
    flops = 2 * (size ** 3) / gpu_time / 1e9
    print(f"GPU matrix multiply time: {gpu_time:.4f} sec")
    print(f"Estimated performance: {flops:.2f} GFLOPS")

except cp.cuda.runtime.CUDARuntimeError as e:
    print("No compatible CUDA GPU detected or drivers not installed.")
